In [2]:
import csv
import os
import numpy as np
from models import CNN_Model, CNN_Model_Mode6

Using TensorFlow backend.


this is a version w/ pli lofz prec, 47 in total !


# process all missense and all cancer

In [3]:
# prefix = '.HIS.'
# # all cancer hostspot and other files
# fins = ['/data/hq2130/large_files/cancer_all.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.GCcorrected.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

In [4]:
# # all cancer hostspot and other files
# fins = ['/data/hq2130/large_files/rare_missense_id.anno.rare.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.GCcorrected.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

# print "done" 

In [5]:
# prefix = '.HS.'

In [ ]:
# # all cancer hostspot and other files
# fins = ['/data/hq2130/large_files/cancer_all.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.GCcorrected.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

In [6]:
# # all cancer hostspot and other files
# fins = ['/data/hq2130/large_files/rare_missense_id.anno.rare.csv']

# fouts = []
# for f in fins:
#     fouts.append(f.split('.csv')[0] + prefix + 'reformat.GCcorrected.csv')
# for fin, fout in zip(fins, fouts):
#     with open(fout, 'w') as fw:
#         w = csv.writer(fw)
#         target_value = 'NA'
#         sel_add_feather(fin, w, wgsa_feat, add_feat, extra_feat, target_value, write_head=True)

# print "done" 

# split the file

In [7]:
pli = np.load('../data/gene/pli.npy').item()
HS_gene = set(gene for gene, pli_score in pli.iteritems() if pli_score < 0.5)
HIS_gene = set(gene for gene, pli_score in pli.iteritems() if pli_score >= 0.5)


# split the files into HS and HIS
fname = '/data/hq2130/large_files/rare_missense_id.anno.rare.All.reformat.csv'
f1 = '/data/hq2130/large_files/rare_missense_id.anno.rare.HIS.reformat.csv'
f2 = '/data/hq2130/large_files/rare_missense_id.anno.rare.HS.reformat.csv'

with open(fname, 'rU') as f:
    r = csv.reader(f)
    head = r.next()
    
    fw1 = open(f1, 'wb')
    w1 = csv.writer(fw1)
    w1.writerow(head)
    
    fw2 = open(f2, 'wb')
    w2 = csv.writer(fw2)
    w2.writerow(head)
            
    for line in r:
        variant = dict(zip(head, line))
        if variant['genename'] in HIS_gene:
            w1.writerow(line)
        if variant['genename'] in HS_gene:
            w2.writerow(line)
        
        
    fw1.close()
    fw2.close()

In [8]:
# split the files
fname = '/data/hq2130/large_files/rare_missense_id.anno.rare.HIS.reformat.csv'
idx, lines_per_file, count = 0, 1350000, 0
with open(fname, 'rU') as f:
    r = csv.reader(f)
    head = r.next()
    for line in r:
        if count == 0:
            outname = fname.split('.csv')[0]+'_idx_'+ str(idx) + '.csv'
            fw = open(outname, 'wb')
            w = csv.writer(fw)
            w.writerow(head)
            w.writerow(line)
            count += 1
        else:
            count += 1
            w.writerow(line)
            if count > lines_per_file:
                fw.close()
                idx += 1
                count = 0
    fw.close()

In [9]:
# split the files
fname = '/data/hq2130/large_files/rare_missense_id.anno.rare.HS.reformat.csv'
idx, lines_per_file, count = 0, 1350000, 0
with open(fname, 'rU') as f:
    r = csv.reader(f)
    head = r.next()
    for line in r:
        if count == 0:
            outname = fname.split('.csv')[0]+'_idx_'+ str(idx) + '.csv'
            fw = open(outname, 'wb')
            w = csv.writer(fw)
            w.writerow(head)
            w.writerow(line)
            count += 1
        else:
            count += 1
            w.writerow(line)
            if count > lines_per_file:
                fw.close()
                idx += 1
                count = 0
    fw.close()

# add MVP annotation

In [10]:

# HIS prediction 
prefix = 'HIS.'
weights_path = '../models/res_HIS-45cols_20170717-01.03.01-weights-improvement-10-0.89.hdf5'
#07-0.94 or 10-0.95 15-0.95 looks fine 07-0.94 looks best 
get_last_layer = False
layer_index = -3 # merge layer, first is negative, 1-4 is positive


dir_all = '/data/hq2130/large_files/'
fins = []
for fname in os.listdir(dir_all):
    if fname.startswith('rare_missense_id.anno.rare.HIS.reformat_idx'):
        fins.append(dir_all + fname)


fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + '.cnn.csv')

for fin, fout in zip(fins, fouts):
    model = CNN_Model_Mode6(weights_path=weights_path, train_flag=False, verbose=0,
                            fname=fin, f_out=fout )
    model.pred(get_last_layer=get_last_layer, layer_index=layer_index)


Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']


models.py:415: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, y], mode="sum")
/home/local/ARCS/hq2130/anaconda2/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
models.py:424: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`
  self.model = Model(input=input_, output=act2)


Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
4

Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
4

/home/local/ARCS/hq2130/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
45 cols used: ['MutationAs

In [12]:
# HS prediction  
prefix = 'HS.'
weights_path = '../models/res_HS-45cols_20170720-23.15.30-weights-improvement-09-0.90.hdf5'
#07-0.94 or 10-0.95 15-0.95 looks fine 07-0.94 looks best 
get_last_layer = False
layer_index = -3 # merge layer, first is negative, 1-4 is positive

dir_all = '/data/hq2130/large_files/'
fins = []
for fname in os.listdir(dir_all):
    if fname.startswith('rare_missense_id.anno.rare.HS.reformat_idx'):
        fins.append(dir_all + fname)


fouts = []
for f in fins:
    fouts.append(f.split('.csv')[0] + '.cnn.csv')

for fin, fout in zip(fins, fouts):
    model = CNN_Model_Mode6(weights_path=weights_path, train_flag=False, verbose=0,
                     fname=fin, f_out=fout )
    model.pred(get_last_layer=get_last_layer, layer_index=layer_index)

Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
4

Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
4

Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
4

Loading training data...
45 cols used: ['MutationAssessor_score_rankscore', 'VEST3_rankscore', 'Polyphen2_HDIV_rankscore', 'SIFT_converted_rankscore', 'PROVEAN_converted_rankscore', 'GenoCanyon_score_rankscore', 'LRT_converted_rankscore', 'Eigen-PC-raw_rankscore', 'Eigen-phred', 'Eigen-PC-phred', 'phyloP20way_mammalian_rankscore', 'GERP++_RS_rankscore', 'SiPhy_29way_logOdds_rankscore', 'phastCons100way_vertebrate_rankscore', 'phyloP100way_vertebrate_rankscore', 'phastCons20way_mammalian_rankscore', 'GM12878_fitCons_score_rankscore', 'HUVEC_fitCons_score_rankscore', 'integrated_fitCons_score_rankscore', 'H1-hESC_fitCons_score_rankscore', 'blosum62', 'pam250', 'SUMO_diff', 'SUMO_score', 'SUMO_cutoff', 'phospho_cutoff', 'phospho_score', 'phospho_diff', 'lofz', 'prec', 'pli', 's_het_log', 'secondary_E', 'secondary_H', 'complex_CORUM', 'preppi_counts', 'gnomad', 'ASA', 'secondary_C', 'gc_content', 'interface', 'ubiquitination', 'BioPlex', 'obs_exp', 'mis_badness']
Loading training data...
4

In [13]:
dir_all = '/data/hq2130/large_files/'
fw = open(dir_all + 'All_rare_missense0724.txt', 'w')
head = ['CHROM', 'POS', 'REF','ALT', 'MVP_score']
fw.write('\t'.join(head) + '\n')
for fname in os.listdir(dir_all):
    if 'cnn' in fname and 'rare' in fname:
        with open(dir_all + fname, 'rU') as f:
            r = csv.reader(f)
            head = r.next()
            for line in r:
                variant = dict(zip(head, line))
                info = [variant['hg19_chr'], variant['hg19_pos(1-based)'], 
                        variant['ref'], variant['alt'], variant['genename'], 
                        variant['cnn_prob']]
                fw.write('\t'.join(info) + '\n')
fw.close()